# Basic test notebook for nuScenes

In [ ]:
from avapi.nuscenes import nuScenesManager


# data_dir = "/data/shared/nuScenes"
data_dir = "/home/data/nuScenes"
NSM = nuScenesManager(data_dir=data_dir, split="v1.0-mini")
NDM = NSM.get_scene_dataset_by_index(0)  # get first scene

## Show camera images

In [ ]:
# show a single image
camera = "CAM_FRONT"
frame_idx = 10
frame = NDM.get_frames(sensor=camera)[frame_idx]
img = NDM.get_image(frame=frame, sensor=camera)
img.view()

In [ ]:
# loop and show all images
frame_idx = 10
frame = NDM.get_frames(sensor=camera)[frame_idx]
cameras = NDM.get_sensor_names_by_type("camera")
for cam in cameras:
    print(f"Showing {cam} camera")
    img = NDM.get_image(frame=frame, sensor=cam)
    img.view()

### Project object location onto image

In [ ]:
import numpy as np
from avstack.geometry.transformations import project_to_image


# get image
camera = "CAM_FRONT"
frame_idx = 10
frame = NDM.get_frames(sensor=camera)[frame_idx]
timestamp = NDM.get_timestamp(frame=frame, sensor=camera)
img = NDM.get_image(frame=frame, sensor=camera)

# get ego location in the future
dt_futures = [2, 4, 6]
positions_future = []
centers_future = []
boxes_future = []
for dt in dt_futures:
    # get the future frame
    frame_future = NDM.get_frame_at_timestamp(
        timestamp=timestamp + dt,
        sensor=camera,
        dt_tolerance=0.25,
    )

    # get the ego
    ego_future = NDM.get_ego(frame=frame_future, agent=None)

    # get boxes future (for debug)
    box_future = ego_future.box.change_reference(img.reference, inplace=False)
    boxes_future.append(box_future)

    # get the future ego agent centers in the current frame
    center_future = ego_future.box.center.change_reference(img.reference, inplace=False)
    centers_future.append(center_future.x)

    # get the future ego agent position in the current frame
    position_future = ego_future.position.change_reference(img.reference, inplace=False)
    positions_future.append(position_future.x)

# get projection location onto the image
positions_on_image = project_to_image(
    pts_3d=np.array(positions_future),
    P=img.calibration.P,
)
centers_on_image = project_to_image(
    pts_3d=np.array(centers_future),
    P=img.calibration.P,
)
print("Positions:\n", positions_on_image)
print("Centers:\n", centers_on_image)

In [ ]:
import cv2
from avapi.visualize.snapshot import show_image, show_image_with_boxes


# debug: show future boxes on image
show_image_with_boxes(img, boxes=boxes_future, inline=True, show=True)

# debug: show waypoints on image
for point_set in [positions_on_image, centers_on_image]:
    img_debug = img.rgb_image.copy()
    for pt in point_set:
        cv2.circle(
            img_debug,
            (int(pt[0]), int(pt[1])),
            radius=20,
            color=(255, 0, 0),
            thickness=-1,
        )
    show_image(img_debug, inline=True)